# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Preliminaries: Loading the data and ML model

In [3]:
dataset = helpers.load_adult_income_dataset()
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
target = dataset["income"]
# Split data into train and test
from sklearn.model_selection import train_test_split
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical=["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical) 
from sklearn.compose import ColumnTransformer

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', LGBMClassifier())])


In [5]:
from dice_ml import Dice
from dice_ml.utils import helpers # helper functions


d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
model = clf.fit(x_train, y_train)
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [40]:
exp = Dice(d, m, method="genetic")
query_instances = x_train[1:2]
e1 = exp.generate_counterfactuals(query_instances, total_CFs=2)
e1.visualize_as_dataframe(show_only_changes=True)

/home/amit/py-envs/env3.8/lib/python3.8/site-packages/sklearn/compose/_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "


Initializing initial parameters to the genetic algorithm...
Initialization complete! Generating counterfactuals...


KeyError: 0

These can now be used to calculate the feature importance scores. 

In [7]:
imp = exp.feature_importance(query_instances, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'workclass': 0.0, 'education': 1.0, 'marital_status': 0.0, 'occupation': 1.0, 'race': 0.0, 'gender': 1.0, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 0.0, 'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.0, 'gender': 1.0, 'age': 1.0, 'hours_per_week': 1.0}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [8]:
imp = exp.feature_importance(query_instances, posthoc_sparsity_param=None)
print(imp.local_importance)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
[{'workclass': 0.8, 'education': 1.0, 'marital_status': 0.5, 'occupation': 0.9, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 0.9, 'education': 0.8, 'marital_status': 0.8, 'occupation': 1.0, 'race': 0.7, 'gender': 0.7, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 0.8, 'education': 0.7, 'marital_status': 1.0, 'occupation': 0.9, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [9]:
cobj=exp.feature_importance(query_instances[0:10], total_CFs=10, posthoc_sparsity_param=None)
print(cobj.summary_importance)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
{'workclass': 0.8333333333333334, 'education': 0.8333333333333334, 'marital_status': 0.7666666666666667, 'occupation': 0.9333333333333333, 'race': 0.5666666666666667, 'gender': 0.5666666666666667, 'age': 1.0, 'hours_per_week': 1.0}


## Convert the counterfactual output to json

In [10]:
json_str = cobj.to_json()
print(json_str)

{
  "cf_examples_list": [
    "{\"model_type\": \"classifier\", \"desired_class\": 1.0, \"desired_range\": null, \"test_instance_df\": \"{\\\"age\\\":{\\\"0\\\":38},\\\"workclass\\\":{\\\"0\\\":\\\"Private\\\"},\\\"education\\\":{\\\"0\\\":\\\"HS-grad\\\"},\\\"marital_status\\\":{\\\"0\\\":\\\"Married\\\"},\\\"occupation\\\":{\\\"0\\\":\\\"Blue-Collar\\\"},\\\"race\\\":{\\\"0\\\":\\\"White\\\"},\\\"gender\\\":{\\\"0\\\":\\\"Male\\\"},\\\"hours_per_week\\\":{\\\"0\\\":44},\\\"income\\\":{\\\"0\\\":0.367}}\", \"final_cfs_df\": \"{\\\"age\\\":{\\\"0\\\":50.0,\\\"1\\\":39.0,\\\"2\\\":51.0,\\\"3\\\":44.0,\\\"4\\\":56.0,\\\"5\\\":48.0,\\\"6\\\":53.0,\\\"7\\\":51.0,\\\"8\\\":51.0,\\\"9\\\":60.0},\\\"workclass\\\":{\\\"0\\\":\\\"Private\\\",\\\"1\\\":\\\"Private\\\",\\\"2\\\":\\\"Other\\\\/Unknown\\\",\\\"3\\\":\\\"Government\\\",\\\"4\\\":\\\"Self-Employed\\\",\\\"5\\\":\\\"Government\\\",\\\"6\\\":\\\"Government\\\",\\\"7\\\":\\\"Self-Employed\\\",\\\"8\\\":\\\"Government\\\",\\\"9\\\":\\\"S

## Convert the json output to a counterfactual object

In [11]:
imp_r=imp.from_json(json_str)
print([o.visualize_as_dataframe() for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

AttributeError: 'NoneType' object has no attribute 'outcome_name'